<a href="https://colab.research.google.com/github/patrickfleith/datapipes/blob/main/Structured_Output_with_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structured Output with Google Gemini
In order to build reliable pipelines in which LLMs consistent return output in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will confirm to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

**To generate structured outputs with Anthropic, we'll use the library `instructor`**.
This will make it very easy to switch between different model providers.

## Setup Google Gemini API

In [4]:
!pip install instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 6.7 MB/s eta 0:00:00


In [5]:
import google.generativeai as genai
import os
from pydantic import BaseModel
from enum import Enum
import instructor

In [6]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [7]:
genai.configure(api_key=GOOGLE_API_KEY)

## Generate JSON
When the model is configured to output JSON, it responds to any prompt with JSON-formatted output.

You can control the structure of the JSON response by supplying a schema. There are two ways to supply a schema to the model:

- As text in the prompt
- As a structured schema supplied through model configuration

Both approaches work in both Gemini 1.5 Flash and Gemini 1.5 Pro.

In [8]:
class City(str, Enum):
    aria = "Aria"
    kniga = "Kniga"
    aquabah = "Aquabah"
    torini = "Torini"

class Character(BaseModel):
    name: str
    age: int
    city: City
    job: str
    two_sentences_background_story: str
    inventory: list[str]

In [9]:
google_client = genai.GenerativeModel(
    model_name="gemini-1.5-flash")

In [10]:
client = instructor.from_gemini(
    client=google_client,
    mode=instructor.Mode.GEMINI_JSON,
)

In [13]:
# note that client.chat.completions.create will also work
player = client.messages.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful table-top RPG gamemaster assistant."},
        {
            "role": "user",
            "content": "Generate an Archer character from Kniga for a low-fantasy RPG campaign. Be creative"},
    ],
    response_model=Character,
)

In [14]:
player

Character(name='Valen', age=24, city=<City.kniga: 'Kniga'>, job='Archer', two_sentences_background_story='Valen grew up in the forests surrounding Kniga, learning to hunt and track from a young age.  His exceptional aim and quiet demeanor led him to join the city guard as an archer, protecting its citizens from both wild beasts and criminal elements.', inventory=['Longbow', 'Quiver of arrows (20)', 'Hunting knife', 'Waterskin', 'Rations (3 days)', 'Leather jerkin'])